In [69]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGSMITH_ENDPOINT"]

'https://api.smith.langchain.com'

## Sequential chain

In [10]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    ChatPromptTemplate)

base_url = "http://localhost:11434/"
model_name = "llama3.2:1b"

llm = ChatOllama(
    base_url = base_url,
    model = model_name,
    temperature = 0.8,
    num_predict = 256
)

system_prompt = SystemMessagePromptTemplate.from_template("You are a {school} teacher. You answer in short sentences.")
human_prompt = HumanMessagePromptTemplate.from_template("Tell me about {topic} in {points} points.")
template = ChatPromptTemplate([system_prompt, human_prompt])
messages = template.invoke({"points": 3, "topic": "Mars", "school": "elementary school"})
response = llm.invoke(messages)
print(response.content)

Here's what I know about Mars:

• Mars is often called the Red Planet because it has reddish colors, which come from iron oxide in the soil.
• Mars has a thin atmosphere and it gets really cold at night - sometimes as low as -125°C (-193°F).
• We have not sent any humans to Mars yet, but there are plans for future missions to explore and possibly even live on the planet.


In [11]:
chain = template | llm
chain

ChatPromptTemplate(input_variables=['points', 'school', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are a {school} teacher. You answer in short sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topic'], input_types={}, partial_variables={}, template='Tell me about {topic} in {points} points.'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', num_predict=256, temperature=0.8, base_url='http://localhost:11434/')

In [12]:
response = chain.invoke({"points": 3, "topic": "Mars", "school": "elementary school"})
print(response.content)

Here's what I know about Mars:

Mars is a big planet. It's closest to Earth.
People want to live on Mars because it's like a new playground.
It takes Mars a long time to get there, so it can't be visited yet!


In [13]:
response

AIMessage(content="Here's what I know about Mars:\n\nMars is a big planet. It's closest to Earth.\nPeople want to live on Mars because it's like a new playground.\nIt takes Mars a long time to get there, so it can't be visited yet!", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-24T03:33:52.7990874Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2335653200, 'load_duration': 22516100, 'prompt_eval_count': 47, 'prompt_eval_duration': 44750100, 'eval_count': 54, 'eval_duration': 2267334500, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-acc2acfa-6dae-4fed-a75f-3369f4a1f77a-0', usage_metadata={'input_tokens': 47, 'output_tokens': 54, 'total_tokens': 101})

### With stroutputparser
To get the content from the response without other metadata.

In [14]:
from langchain_core.output_parsers import StrOutputParser

chain = template | llm | StrOutputParser()

response = chain.invoke({"points": 3, "topic": "Mars", "school": "elementary school"})
print(response)

Mars is a planet, it's red and rocky. 

1. Scientists think there might have been water on Mars a long time ago.

2. Mars has big valleys called Valles Marineris, they're really deep.

3. There's evidence of lakes and rivers on Mars, but now they've dried up.


In [17]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434/"
model_name = "llama3.2:1b"

llm = ChatOllama(
    base_url = base_url,
    model = model_name
)

system_prompt = SystemMessagePromptTemplate.from_template("You are a {school} teacher. You answer in short sentences.")
human_prompt = HumanMessagePromptTemplate.from_template("Tell me about {topic} in {points} points.")
template = ChatPromptTemplate([system_prompt, human_prompt])

chain = template | llm | StrOutputParser()

response = chain.invoke({"points": 3, "topic": "Mars", "school": "elementary school"})
print(response)

Here's what I know about Mars:

* Mars is the fourth planet from the Sun and it's really, really far away.
* Scientists think that Mars might have had water on it a long time ago, but now it's mostly dry because of the cold temperatures.
* There are volcanoes on Mars and rocks on its surface that are shaped like craters - it's pretty cool!


### Multiple chains

In [ ]:
analyze_prompt = ChatPromptTemplate.from_template("""
Analyze this text: 
{response}

You need to tell me how difficult it is to understand.
Answer me in one sentence only.
""")

In [27]:
chain2 = analyze_prompt | llm | StrOutputParser()
response = chain2.invoke({"response": response})
print(response)

The difficulty level of the text in understanding its content is moderate.


In [28]:
composed_chain = {"response": chain} | chain2
response = composed_chain.invoke({"points": 3, "topic": "Mars", "school": "elementary school"})
print(response)

The text requires a basic level of knowledge about Mars, covering its physical features, geological history, and scientific significance, making it moderately accessible but not extremely complex.


## Parallel chains

In [29]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434/"
model_name = "llama3.2:1b"

llm = ChatOllama(
    base_url = base_url,
    model = model_name
)

In [30]:
system_prompt = SystemMessagePromptTemplate.from_template("You are a helpful assistant. You answer in short sentences.")
human_prompt = HumanMessagePromptTemplate.from_template("Tell me facts about {topic} in {points} points.")
template = ChatPromptTemplate([system_prompt, human_prompt])

fact_chain = template | llm | StrOutputParser()

response = fact_chain.invoke({"points": 3, "topic": "solar system"})
print(response)

Here are three key facts about the solar system:

1. **The Sun is at the center**: The Sun is the star at the center of our solar system, and it's what makes all the planets warm and habitable.
2. **The planets have unique features**: Each planet in our solar system has its own distinct characteristics, such as Jupiter's massive size or Earth's ability to support life.
3. **Asteroids and comets are also part of the solar system**: Asteroids are rocky objects that orbit around the Sun, while comets are icy bodies that release gas and dust as they approach the inner Solar System.


In [31]:
human_prompt = HumanMessagePromptTemplate.from_template("Write me a poem about {topic} in {lines} lines.")
template = ChatPromptTemplate([system_prompt, human_prompt])

poem_chain = template | llm | StrOutputParser()

response = poem_chain.invoke({"lines": 2, "topic": "solar system"})
print(response)

The sun at the center shines bright,
A solar system spinning with equal might.


In [32]:
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(fact=fact_chain,
                                  poem=poem_chain)
parallel_chain

{
  fact: ChatPromptTemplate(input_variables=['points', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant. You answer in short sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topic'], input_types={}, partial_variables={}, template='Tell me facts about {topic} in {points} points.'), additional_kwargs={})])
        | ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434/')
        | StrOutputParser(),
  poem: ChatPromptTemplate(input_variables=['lines', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant. You answer in short sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(in

In [ ]:
response = parallel_chain.invoke({
  "points": 3,
  "lines": 2, 
  "topic": "solar system"
})

print("Fact chain output:")
print(response['fact'])
print()
print("Poem chain output:")
print(response['poem'])


Fact chain output:
Here are three interesting facts about the solar system:

• The sun is the center of our solar system, and it makes up about 99.8% of its total mass.
• Pluto was previously considered the ninth planet in our solar system but has been reclassified as a dwarf planet by the International Astronomical Union (IAU).
• Venus rotates in the opposite direction of most other planets in our solar system, a phenomenon known as retrograde rotation.

Poem chain output:
The sun at center, shining bright
Planets orbit around, in celestial delight


## Chain Router
The router chain is used to route the output of a previous runnable to the next runnable based on the output of the previous runnable.

In [35]:
prompt = """
Given the user review below, classify it as either being about "Positive" or "Negative".
Do not respond with more than one word.

Review: {review}
Classification:
"""

template = ChatPromptTemplate.from_template(prompt)

chain = template | llm | StrOutputParser()
review = "I am not happy with the service. It is not good."
chain.invoke({'review': review})

'Negative'

In [48]:
positive_prompt = """
You are expert in writing reply for positive reviews.
You need to encourage the user to share their experience on social media.
Review: {review}
Answer:
"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()

In [49]:
negative_prompt = """
You are expert in writing reply for negative reviews.
You need first to apologize for the inconvenience caused to the user.
You need to encourage the user to share their concern on following email:'farrel@gmail.com'.
Review: {review}
Answer:
"""

negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()

In [50]:
def route(info):
  sentiment = info["sentiment"].lower()
  if "positive" in sentiment:
    return positive_chain
  else:
    return negative_chain

In [51]:
route({"sentiment": "Positive"})

ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template='\nYou are expert in writing reply for positive reviews.\nYou need to encourage the user to share their experience on social media.\nReview: {review}\nAnswer:\n'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434/')
| StrOutputParser()

In [52]:
from langchain_core.runnables import RunnableLambda

full_chain = {"sentiment": chain, "review": lambda x: x["review"]} | RunnableLambda(route)

In [53]:
# review = "Thank you so much for providing such a great plateform for learning. I am really happy with the service."
review = "I am not happy with the service. It is not good."

response = full_chain.invoke({"review": review})
print(response)

Here's a potential response:

"I'm so sorry to hear that you're not satisfied with our service. I can only imagine how frustrating it must be when things don't meet your expectations. Please know that we take all complaints seriously and are truly disappointed to hear that we fell short of your standards.

I'd like to help resolve this issue, if there's anything specific that didn't meet your expectations or if you could provide more details about the problem, I'd be more than happy to listen and assist in any way possible. Your satisfaction is our top priority, and I'm committed to making things right.

If you have the time, could you please contact me directly at farrel@gmail.com so we can discuss this further? We value your feedback and would love the opportunity to make things right. Thank you for sharing your concerns with us."

This response:

1. Apologizes sincerely for the inconvenience caused
2. Acknowledges the user's frustration and disappointment
3. Expresses empathy toward

## Custom runnable chain with RunnablePassthrough and RunnableLambda

In [54]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [55]:
def char_count(text):
  return len(text)

def word_count(text):
  return len(text.split())

In [58]:
prompt = ChatPromptTemplate.from_template("Explain these inputs: {input1} and {input2} in three sentences.")

In [59]:
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"input1": "Earth", 
                         "input2": "Mars"})
print(response)

Here are three sentences explaining Earth and Mars:

Earth is the third planet from the Sun, a terrestrial body with a solid surface that supports life, featuring diverse climates, oceans, and landscapes, including continents, mountains, and vast deserts. Mars, on the other hand, is a rocky, barren world with a thin atmosphere, also known as the Red Planet due to its reddish appearance caused by iron oxide in the soil, with evidence of past water activity and potential habitability. Both Earth and Mars have captivated human curiosity and exploration efforts, with ongoing research and missions seeking to understand their unique characteristics and potentially one day establish human settlements or other life-supporting systems.


In [61]:
chain = prompt | llm | StrOutputParser() | {"char_count": RunnableLambda(char_count),
                                            "word_count": RunnableLambda(word_count),
                                            "output": RunnablePassthrough()}

response = chain.invoke({"input1": "Earth", 
                         "input2": "Mars"})
print(response)

{'char_count': 590, 'word_count': 91, 'output': 'Here are three sentences explaining the relationship between Earth and Mars:\n\nEarth is a terrestrial planet, meaning it has a solid surface and atmosphere, while Mars is also a rocky planet with its own thin atmosphere. Both planets share some similarities, such as their similar size and mass, but they have distinct differences in terms of their composition, geology, and atmospheric conditions. Despite being separated by over 140 million miles, Earth and Mars have been extensively studied by spacecraft, providing valuable insights into the formation and evolution of our solar system.'}


In [64]:
print("Output:")
print(response['output'])
print()
print(f"Char count: {response['char_count']}")
print(f"Word count: {response['word_count']}")

Output:
Here are three sentences explaining the relationship between Earth and Mars:

Earth is a terrestrial planet, meaning it has a solid surface and atmosphere, while Mars is also a rocky planet with its own thin atmosphere. Both planets share some similarities, such as their similar size and mass, but they have distinct differences in terms of their composition, geology, and atmospheric conditions. Despite being separated by over 140 million miles, Earth and Mars have been extensively studied by spacecraft, providing valuable insights into the formation and evolution of our solar system.

Char count: 590
Word count: 91


## Custom chain with @chain decorator

In [65]:
from langchain_core.runnables import chain

In [66]:
@chain
def custom_chain(params):
  return {
    "fact": fact_chain.invoke(params),
    "poem": poem_chain.invoke(params)
  }

In [68]:
params = {
  "points": 3,
  "lines": 2, 
  "topic": "solar system"
}

response = custom_chain.invoke(params)

print("Fact chain output:")
print(response['fact'])
print()
print("Poem chain output:")
print(response['poem'])


Fact chain output:
Here are three key facts about the solar system:

1. Our solar system has eight planets, plus dwarf planets and other smaller bodies like asteroids and comets.
2. The Sun is at the center of our solar system, providing light and heat to the planets, which are held in orbit around it due to gravity.
3. Pluto was previously considered the ninth planet but was reclassified as a dwarf planet by the International Astronomical Union in 2006.

Poem chain output:
The sun is the center bright,
Stars and planets dance through night.
